In [1]:
import numpy as np
import pandas as pd
from astropy.io import fits, ascii
from astropy.table import Table

import matplotlib.pyplot as plt
from matplotlib import rc
rc('text', usetex=True)
rc('xtick', labelsize=20)
rc('ytick', labelsize=20)

import altair as alt

In [10]:
data = Table.read('catalogs/cantat_gaudin_2020_cluster_members.fits.gz')
len(data)

234128

In [15]:
# cat = ascii.read('http://cdsarc.u-strasbg.fr/viz-bin/nph-Cat/html?J/A+A/640/A1/table1.dat').to_pandas()
cat = pd.read_csv("catalogs/cantat_gaudin_2020_cluster_summary.csv.gz")
len(cat)

1803

In [137]:
sel = data[data['proba'] > .95].to_pandas()
sources = sel.drop(columns=['_RAJ2000', '_DEJ2000', 'RA_ICRS', 'DE_ICRS', 'pmRA_',
       'pmDE', '_RA_icrs', '_DE_icrs', 'Teff50', 'RV', 'o_Gmag'])
cnames = np.array([cc.strip() for cc in sources['Cluster']])
sources['Cluster'] = np.array(cnames, dtype=str)
sources

,Cluster,GaiaDR2,Plx,Gmag,BP-RP,proba
0,Melotte_25,36629890579758592,28.598377,12.626208,2.572736,1.0
1,Melotte_25,38329666836450304,21.946148,15.111156,3.214545,1.0
2,Melotte_25,38354680725946240,21.721823,9.766113,1.295699,1.0
3,Melotte_25,38670343642226816,24.153702,14.832144,3.200334,1.0
4,Melotte_25,38929141191925760,21.241981,13.329974,2.986785,1.0
...,...,...,...,...,...,...
123492,UPK_51,6910572932968568448,1.662175,15.873636,1.497299,1.0
123493,UPK_51,6910576059704797184,1.857415,16.018810,1.762252,1.0
123494,UPK_51,6913214823187911808,1.821802,16.425526,1.668467,1.0
123495,UPK_51,6913597556312267648,1.864715,14.675908,1.137109,1.0


In [156]:
dist = np.array(cat['DistPcpc'], dtype=float)
age = np.array(cat['AgeNN[yr]'], dtype=float)
name = np.array(cat['Cluster---'], dtype=str)
nstar = np.array(cat['nbstars07---'], dtype=int)
radius = np.array(cat['r50arcsec'], dtype=float)

sum_df = pd.DataFrame(data={'Dist (pc)': dist, 
                        'log(age)': age, 
                        'age': 10**sum_df['log(age)']/1e6,
                        'Cluster': name,
                        'Nstar': nstar, 
                        'Radius (arcsec)': radius, 
                        'log(Nstar)': np.log10(nstar)})
print(len(sources))

df = sources.merge(sum_df, on='Cluster')
df[df['Dist (pc)'] < 200]

123497


,Cluster,GaiaDR2,Plx,Gmag,BP-RP,proba,Dist (pc),log(age),age,Nstar,Radius (arcsec),log(Nstar)
0,Melotte_22,51616268426009472,7.440226,13.991986,1.996518,1.0,128.0,7.89,77.624712,952,1.274,2.978637
1,Melotte_22,51694746068624128,7.226617,11.226496,1.038179,1.0,128.0,7.89,77.624712,952,1.274,2.978637
2,Melotte_22,53335045618385536,7.422380,13.766633,1.909817,1.0,128.0,7.89,77.624712,952,1.274,2.978637
3,Melotte_22,63289920815872256,7.284577,11.309966,1.056929,1.0,128.0,7.89,77.624712,952,1.274,2.978637
4,Melotte_22,63436396380626688,7.492297,14.869678,2.444868,1.0,128.0,7.89,77.624712,952,1.274,2.978637
...,...,...,...,...,...,...,...,...,...,...,...,...
87617,NGC_2451A,5587909961440258560,5.156176,17.013191,2.892288,1.0,195.0,7.55,35.481339,331,1.146,2.519828
87618,NGC_2451A,5587987477010155520,4.989618,17.784435,3.199818,1.0,195.0,7.55,35.481339,331,1.146,2.519828
87619,NGC_2451A,5588125637517514496,5.010439,16.247124,2.676545,1.0,195.0,7.55,35.481339,331,1.146,2.519828
87620,NGC_2451A,5588232152708295424,5.072148,15.605858,2.501937,1.0,195.0,7.55,35.481339,331,1.146,2.519828


In [173]:
source = sum_df

alt.Chart(source).mark_circle(size=60).properties(
    width=800,
    height=500
).encode(
    x='Dist (pc)',
    y='age',
    color=alt.Color('log(Nstar)', scale=alt.Scale(scheme='greenblue')),
    tooltip=['Cluster', 'age', 'Nstar', 'Radius (arcsec)']
).interactive()

alt.Chart(...)

In [174]:
source = df[df['Cluster'] == 'Melotte_22']

alt.Chart(source).mark_circle(size=60).properties(
    width=800,
    height=500
).encode(
    x='BP-RP',
    y=alt.Y('Gmag', scale=alt.Scale(domain=(19,4))),
    color=alt.Color('proba', scale=alt.Scale(scheme='greenblue')),
    tooltip=['GaiaDR2', 'proba', 'Plx']
)

alt.Chart(...)